# Adding a Slider to the Exoplanets Census  

In the last part of this episode, we will finish our copy of the  _Exoplanet Census_ plot that can be found at the [Exoplanet's NASA site](https://exoplanets.nasa.gov/discovery/discoveries-dashboard/):

<img src="../img/Exoplanet Census.png" alt="Exoplanet Census" width="400"/>

We learnt how to process the data with the `FSharp.Data` library, and how to plot it with `Plotly`, and now we need to add a slider to the plot, so we can see which exoplanets were discovered up to a given year. 

Let us start by opening all the libraries and read and process our data:

In [1]:
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: FSharp.Data"

Installed Packages FSharp.Data, 6.3.0 Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.0

Loading extensions from `/Users/flavioc/.nuget/packages/plotly.net.interactive/4.2.0/lib/netstandard2.1/Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

In [3]:
[<Literal>]
let exoplanetsFile = "../data/consolidatedExoplanets.csv"

type ExoPlanetType = FSharp.Data.CsvProvider<exoplanetsFile, HasHeaders=true, PreferOptionals=true>
let planets = ExoPlanetType.GetSample()

We defined a function to get the year of discovery

In [4]:
let getYearOfDiscovery (ref: string) = 
    let found = RegularExpressions.Regex.Matches(ref, " \d{4}")
    match found.Count with
    | 0 -> None
    | 1 -> Some (found.[0].Value.Trim(' ') |> int) 
    | _ -> failwith "More than one year found"
    

Then a useful type for the data

In [6]:
type ExoCensusData =
    {
        DiscoveryMethodName : string 
        OrbitTimes: decimal 
        Masses: decimal  
        YearOfDiscovery: int 
    }

And finally created a sequence of it. We used a tuple to save the intermediate data from the table, since there are some `option` values, and then, we filtered them to our `ExoCensusData`:

In [7]:
let data =
    planets.Rows 
    |> Seq.map (fun row -> (row.Discoverymethod, row.Pl_orbper, row.Pl_masse, getYearOfDiscovery row.Disc_refname))
    |> Seq.choose (fun (method, period, mass, year) -> 
        match period,mass,year with
        | Some p, Some m, Some y -> Some {DiscoveryMethodName = method; OrbitTimes = p; Masses = m; YearOfDiscovery = y}
        | _ -> None)
    

In [ ]:
data.DisplayTable()

We grouped our data by discovery method

In [8]:
let dataByDiscoveryMethod = 
    data 
    |> Seq.groupBy (fun exoData -> exoData.DiscoveryMethodName)

and then created the plot type and plot data:

In [9]:
type ExoTrace = 
    {
        DiscoveryMethodName : string 
        OrbitTimes : seq<decimal>
        Masses : seq<decimal>
        }


In [10]:
let exoTraces = 
    dataByDiscoveryMethod
    |> Seq.map (fun (method, data) -> 
            let orbits = data |> Seq.map (fun exoData -> exoData.OrbitTimes)
            let masses = data |> Seq.map (fun exoData -> exoData.Masses)
            {DiscoveryMethodName = method; OrbitTimes = orbits; Masses = masses})

We also defined several values to customize de layout and the configuration of the plot:

In [11]:
open Plotly.NET.LayoutObjects // this namespace contains all object abstractions for layout styling

let orbPeriodAxis =
    LinearAxis.init (
        Title = Title.init (Text = "ORBIT PERIOD (EARTH DAYS)"),
        AxisType = StyleParam.AxisType.Log,
        ShowLine = true,
        ShowGrid = false,
        Range = StyleParam.Range.MinMax (-2, 8),
        Ticks = StyleParam.TickOptions.Outside
    )

let massLogAxis =
    LinearAxis.init (
        Title = Title.init (Text = "PLANET MASS (EARTH MASSES)"),
        AxisType = StyleParam.AxisType.Log,
        ShowLine = true,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Outside
    )

let openCircle = 
    StyleParam.MarkerSymbol.Modified(
            StyleParam.MarkerSymbol.Circle,
            StyleParam.SymbolStyle.Open
        )  

In [12]:
open Plotly.NET.ConfigObjects

let layout =
    Layout.init(
                Width = 1000,
                Height = 500
    )


Finally, the previous chart for all the data is:

In [13]:
exoTraces
|> Seq.map (fun exo -> 
                Chart.Point(exo.OrbitTimes,exo.Masses, Name = exo.DiscoveryMethodName)
                |> Chart.withMarkerStyle(Symbol=openCircle))
|> Chart.combine
|> Chart.withXAxis orbPeriodAxis
|> Chart.withYAxis massLogAxis
|> Chart.withLayout layout 

<!-- Plotly chart will be drawn inside this DIV -->

### Adding the year Slider 

One of the possibilities that Plotly gives us is to add a slider to change the plot interactively. The Exoplanet census plot has a slider that selects the number of planets according to the year of the discovery. 
Let us think about this type for a minute. We are working on a plot that will show the exoplanets discovered up to a certain year. Therefore, we need to be able to filter the data according to that condition _before_ building the traces to plot. 

So we need to be able to filter the data by year of discovery. Let us create the range of years:

In [14]:
let yearsOfDiscovery = [1989..1..2023]
yearsOfDiscovery

[ 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008 ... (more) ] HeadOrDefault 1989 TailOrNull [ 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009 ... (more) ] HeadOrDefault 1990 TailOrNull [ 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010 ... (more) ] HeadOrDefault 1991 TailOrNull [ 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011 ... (more) ] HeadOrDefault 1992 TailOrNull [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] HeadOrDefault 1993 TailOrNull [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] Head 1993 Tail [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] (values) [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] Head 1992 Tail [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] HeadOrDefault 1993 TailOrNull [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] Head 1993 Tail [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] (values) [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] (values) [ 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011 ... (more) ] Head 1991 Tail [ 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011 ... (more) ] HeadOrDefault 1992 TailOrNull [ 1

Now we compute the data for each traces. We define a function `exoTracesUpToYear` that receives a year and creates the corresponding data for the traces.

In [15]:
let exoTracesUpToYear year =
    dataByDiscoveryMethod
    |> Seq.map (fun (method, data) -> 
            let dataUpToYear = 
                data 
                |> Seq.filter (fun exoData -> exoData.YearOfDiscovery <= year)

            let orbits = 
                dataUpToYear
                |> Seq.map (fun exoData -> exoData.OrbitTimes)
            let masses = 
                dataUpToYear 
                |> Seq.map (fun exoData -> exoData.Masses)
            {DiscoveryMethodName = method; OrbitTimes = orbits; Masses = masses}
            )
    |> Seq.sortBy (fun exo -> exo.DiscoveryMethodName)

How many traces we have for each year?

In [16]:
yearsOfDiscovery
    |> Seq.map (fun year -> year,exoTracesUpToYear year |> Seq.length)
    |> Seq.iter (fun (year,count) -> printfn "%d,%d" year count)

1989,9
1990,9
1991,9
1992,9
1993,9
1994,9
1995,9
1996,9
1997,9
1998,9
1999,9
2000,9
2001,9
2002,9
2003,9
2004,9
2005,9
2006,9
2007,9
2008,9
2009,9
2010,9
2011,9
2012,9
2013,9
2014,9
2015,9
2016,9
2017,9
2018,9
2019,9
2020,9
2021,9
2022,9
2023,9


The way we are building our set of traces defines `seq []` (an empty sequence) for discovery methods that were not available at a given year. Therefore, we have nine traces for each data up to a given year. Let us define a value that will become handy in a moment:

In [17]:
let numberOfDiscoveryMethods = 9 

Inspecting the [Slider example](https://plotly.net/chart-layout/sliders.html), one has to make a chart with all the possible traces the slider will accommodate, and then make visible the chart for the selected year in the slider, that is, we need to select only the traces for a given year. 

To connect a chart with the slider, we need to create an object `Slider` through the function `Slider.init` of Plotly. 
One of the arguments that receives this constructor is `Steps`, which is a `seq` of `SliderStep` objects. This object carries a `Method` that describes how to update the plot when the slider moves, a `Label` to print the current value of the slider, and `Args` that are the arguments that are passed to the method `Method`. One of the arguments is `visible` that determines which traces are visible for each position of the slider. 

> Note that this does not look too fsharpy, but bear in mind that Plotly is originally a JavaScript library, and the specification of each plot is a JSON file.

> This [answer](https://community.plotly.com/t/multiple-traces-with-a-single-slider-in-plotly/16356/2) (although in Python) is extremely helpful to understand how the slider works for multiple traces.

To make it work properly, one has to create a plot with _all_ the traces one wants to show, and select them accordingly with the `visible` argument. Then, we need to plot nine traces for each year, for all the years since 1989 (35), for a total of 9 x 35 = 315 traces. Since for each year we have nine traces to show, `visible` will be a mask (implemented as a sequence of booleans), being false for all traces, except the nine ones to be shown for a given year.

Let us start by creating an array of `numberOfDiscoveryMethods * (yearsOfDiscovery |> Seq.length)` booleans set to `false`:

In [18]:
let visibleInit = 
    Array.create (numberOfDiscoveryMethods * (yearsOfDiscovery |> Seq.length)) false    

and define a function that returns the `visible` mask as a sequence for a given year:

In [19]:
let setVisibleForYear year = 
    let i0 = year - 1989 
    visibleInit
    |> Seq.mapi (fun i visible -> 
                    if i >= (i0 * numberOfDiscoveryMethods) && 
                       i <  ((i0 + 1) * numberOfDiscoveryMethods) then true
                    else false 
                )
    |> Array.ofSeq
                    

So, for example, for 1989, the mask will start with nine consecutive `true`s, followed by all `false`s:

In [20]:
let visibility1989 = setVisibleForYear 1989
visibility1989.DisplayTable()

value
True
True
True
True
True
True
True
True
True
False


For 1990, the mask will start with nine consecutive `false`s, followed by nine `trues`s and continuing with all falses:

In [21]:
let visibility1990 = setVisibleForYear 1990
visibility1990.DisplayTable()

value
False
False
False
False
False
False
False
False
False
True


And so on. Now we can build the `SliderStep` array, following the example in the Plotly documentation:

In [22]:
let sliderSteps =
    yearsOfDiscovery
    |> Seq.indexed
    |> Seq.map (fun (i, year) ->
        // Create a visibility and a title parameter
        // The visibility parameter includes an array where every parameter
        // is mapped onto the trace visibility
        let visible = setVisibleForYear year |> box        
        let title = sprintf "Year: %d" year |> box

        SliderStep.init (
            Args = [ "visible", visible; "title", title ],
            Method = StyleParam.Method.Update,
            Label = string (year)
        ))
        

Once we got this sequence, we can build the `Slider`` object:

In [23]:
let slider =
    Slider.init (
        CurrentValue = SliderCurrentValue.init (Prefix = "Year: "),
        Padding = Padding.init (T = 50),
        Steps = sliderSteps
    )        

We are almost there! Now we run all over the possible values of the slider (it is indexed from 0 to `yearsOfDiscovery.Length - 1`), create the charts for each trace and format all of them with our choice of axis and layout as before:

In [24]:
let exoCensusChart =
    Seq.init yearsOfDiscovery.Length (fun i -> i)
    |> Seq.map (fun yearIdx ->        
        // Some plot must be visible here or the chart is empty at the beginning
        let chartVisibility = 
            if yearIdx = 0 then
                StyleParam.Visible.True
            else
                StyleParam.Visible.False

        let go =
            (yearIdx + 1989)
            |> exoTracesUpToYear
            |> Seq.map (fun exo -> 
                            Chart.Point(exo.OrbitTimes,exo.Masses, Name = exo.DiscoveryMethodName)
                            |> Chart.withMarkerStyle(Symbol=openCircle)
                            |> Chart.withTraceInfo (Visible = chartVisibility))
            |> Chart.combine            

        go) 
    |> GenericChart.combine
    |> Chart.withXAxis orbPeriodAxis
    |> Chart.withYAxis massLogAxis
    |> Chart.withLayout layout 

As a side note we use the `chartVisibility` value to clear the chart for 1989. Now, we pass the `scatterChart` and the  `Slider` to `Chart.withSlider` to create the complete chart:

In [25]:
let chart = exoCensusChart |> Chart.withSlider slider

In [26]:
chart

<!-- Plotly chart will be drawn inside this DIV -->

And voila! Our beautiful exoplanet census! Move the slider to see the discoveries appearing!

## Final words

Finally, our example is completed, we went from processing the data with Type providers to our exoplanet census plot! This showcases a typical workflow for data processing in F#. Along the way, we reviewed how to use the powerful Type Providers to manage structured information, and even used it to create and save massaged data into a file.

Plotly is a wonderful plotting library, which is tailored for the Web. In fact, one can create an html with:


In [ ]:
chart |> Chart.saveHtml "exoplanets census"

and open it with a browser.